In [2]:
 import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils
from pathlib import Path
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from lockdowndates.core import LockdownDates

from catboost import CatBoostRegressor
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates
from datetime import timedelta, date

In [18]:
X, y = utils.get_train_data()
X_final_test = pd.read_parquet(Path("data") / "final_test.parquet")

In [19]:
X_weather = utils.create_x_weather(X)
X_ld = utils.generate_lockdown_dates(X_weather)
X = utils.is_jour_ferie(X_ld)


In [20]:
X = X.drop(columns=['date_merge','counter_technical_id','coordinates','counter_installation_date',
                'site_name','site_id', 'counter_name', 'counter_id','index'])
X

,date,latitude,longitude,temp,precip,windspeed,visibility,icon_encoded,Lockdown,is_ferie,is_vacances
0,2020-09-01 01:00:00,48.840801,2.333233,16.1,0.00,13.6,25.1,3,1,0,0
1,2020-09-01 01:00:00,48.840801,2.333233,16.1,0.00,13.6,25.1,3,1,0,0
2,2020-09-01 01:00:00,48.853720,2.357020,16.1,0.00,13.6,25.1,3,1,0,0
3,2020-09-01 01:00:00,48.853720,2.357020,16.1,0.00,13.6,25.1,3,1,0,0
4,2020-09-01 01:00:00,48.885290,2.326660,16.1,0.00,13.6,25.1,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
534976,2021-09-09 23:00:00,48.842010,2.367290,22.1,2.54,16.6,13.7,1,1,0,0
534977,2021-09-09 23:00:00,48.864620,2.314440,22.1,2.54,16.6,13.7,1,1,0,0
534978,2021-09-09 23:00:00,48.864620,2.314440,22.1,2.54,16.6,13.7,1,1,0,0
534979,2021-09-09 23:00:00,48.848400,2.275860,22.1,2.54,16.6,13.7,1,1,0,0


In [22]:
from sklearn.preprocessing import MinMaxScaler


features_scale = ['temp',	'precip',	'windspeed',	'visibility',	'icon_encoded',	'Lockdown',	'is_ferie',	'is_vacances']



col_transformer = ColumnTransformer(
    transformers=[
        
        ('num', MinMaxScaler(), features_scale)
    ],
    remainder='passthrough'  # This keeps all other columns unchanged
)

pipeline = Pipeline([
        ('minmax', col_transformer)
])

pd.DataFrame(pipeline.fit_transform(X))

,0,1,2,3,4,5,6,7,8,9,10
0,0.648649,0.0,0.215847,0.974619,0.75,0.0,0.0,0.0,2020-09-01 01:00:00,48.840801,2.333233
1,0.648649,0.0,0.215847,0.974619,0.75,0.0,0.0,0.0,2020-09-01 01:00:00,48.840801,2.333233
2,0.648649,0.0,0.215847,0.974619,0.75,0.0,0.0,0.0,2020-09-01 01:00:00,48.85372,2.35702
3,0.648649,0.0,0.215847,0.974619,0.75,0.0,0.0,0.0,2020-09-01 01:00:00,48.85372,2.35702
4,0.648649,0.0,0.215847,0.974619,0.75,0.0,0.0,0.0,2020-09-01 01:00:00,48.88529,2.32666
...,...,...,...,...,...,...,...,...,...,...,...
496822,0.851351,0.077933,0.297814,0.395939,0.25,0.0,0.0,0.0,2021-09-09 23:00:00,48.84201,2.36729
496823,0.851351,0.077933,0.297814,0.395939,0.25,0.0,0.0,0.0,2021-09-09 23:00:00,48.86462,2.31444
496824,0.851351,0.077933,0.297814,0.395939,0.25,0.0,0.0,0.0,2021-09-09 23:00:00,48.86462,2.31444
496825,0.851351,0.077933,0.297814,0.395939,0.25,0.0,0.0,0.0,2021-09-09 23:00:00,48.8484,2.27586


In [32]:
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np

# Parameters for TimeseriesGenerator
n_input = 3  # Length of the output sequences (in number of timesteps)
n_features = X_scaled_df.shape[1]  # Number of features

# Generate data for the LSTM
generator = TimeseriesGenerator(X_scaled_df.values, y_scaled, length=n_input, batch_size=1)


AttributeError: partially initialized module 'keras.src' has no attribute 'utils' (most likely due to a circular import)